Following the tutorial from: https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy on the Kaggle titanic problem https://www.kaggle.com/c/titanic. I am using this a tutorial as a refresher of the practical aspect of machine learning and data science to unsure that I do not lose my skills from my masters degree. It will aslo allow me to see a project tackled from another perspective and hopefully allow me to learn something new

# 1. Defining the problem
It is a binary supervised learning task that looks to predict if an individual will survive the sinking of HMS Titanic. 

__Project summary:__ The sinking of the RMS Titanic is one of the most infamous shipwrecks in history. On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.vOne of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

# 2. Gathering the data
The date for this problem is not include within this repository but can be found here: https://www.kaggle.com/c/titanic/data instead

# 3. Prepare the data
The data is already in .csv format and is deliberately in an easy to use format so the only requirement here is data cleaning. 